# Script to create all required outputs to run InVEST model for a given AOI!

*Author: Jack Beard, FutureWater*

Suggested steps before running:
1. Install Anaconda from https://www.anaconda.com/
2. Open "Anaconda prompt"
3. Create a new environment (called gee or similar) by running:
    > conda create -n gee python=3.7
4. Activate environment by running 
    > conda activate gee
5. Install required packages by running:
    
    > conda install -c conda-forge earthengine-api
    
    > conda install geemap -c conda-forge
    
    > conda install geopandas
    
    > conda install -c conda-forge rasterio
   
6. Open Jupyter Notebook by running:
    > jupyter notebook


*Potential improvements:*
1. Some sort of automation of creation of climate zones using precip/temp datasets for larger areas
2. Ability to choose between datasets

In [3]:
# Importing required libraries

import ee
import geemap

import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()


Enter verification code: 4/1ARtbsJrq6aW4cZeSW4rLw0pnEu4PzY07TNwOUwzFrUyGoQDfuLqoLsfowwc

Successfully saved authorization token.


## Downloading background datasets

### Defining input shapefile and input paths

In [43]:
path_wd = "c:/Work/FW/InVEST/Lukanga_Wetland_Subbasins/GEE_files/"
shapefile = "c:/Work/FW/InVEST/Lukanga_Wetland_Subbasins/AOI/Wetland_Subbasin.shp" # At tis point, this needs to be in WGS84!

project_crs = "EPSG:32735" # define espg for reprojection of output rasters

### AOI data

In [44]:
## # Using admin layer
## country_name = "Zambia"
## province_name = "Western"
## # Importing country shape - could also use user asset
## shp = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq('ADM0_NAME', country_name)).filter(ee.Filter.eq('ADM1_NAME', province_name))

shp = geemap.shp_to_ee(shapefile)

shp_geom = shp.geometry()

# Creating map
Map = geemap.Map()
Map.centerObject(shp)
Map.addLayerControl()

#Add shapefile to Map  
Map.addLayer(shp, {},'AOI')

Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

### Land use data

In [31]:
# https://developers.google.com/earth-engine/datasets/catalog/ESA_WorldCover_v100
LU = ee.ImageCollection("ESA/WorldCover/v100") 
LU_reproj = LU.filterBounds(shp_geom).first() \
.setDefaultProjection(project_crs)

LU_aoi = LU_reproj.clip(shp_geom)

#visualization = {bands: ['Map']}

# Adding to above map
Map.addLayer(LU_aoi, {}, "Landcover")

# Getting projection and transform to create common gridding system!
projection = LU_aoi.projection().getInfo()

# Export to file
geemap.ee_export_image(LU_aoi.clip(shp_geom),
                       path_wd + "lu_aoi.tif",
                       scale = 100,
                       region = shp_geom,
                       crs = project_crs)

Generating URL ...
Please wait ...
Data downloaded to c:\Work\FW\InVEST\Lukanga_Wetland_Subbasins\GEE_files\lu_aoi.tif


### Soil Type Data

In [45]:
# Using FutureWater's OWN dataset (hihydrosoil) - https://samapriya.github.io/awesome-gee-community-datasets/projects/hihydro_soil/
# The dual soil class groups are coded as 14 (A/D), 24 (B/D) and 34 (C/D) - Remapping them for visualization
hydrologic_soil_group = ee.Image('projects/sat-io/open-datasets/HiHydroSoilv2_0/Hydrologic_Soil_Group_250m') \
.remap([1, 14, 2, 24, 3, 34, 4], [1, 1, 2, 2, 3, 3, 4]) \
.setDefaultProjection(project_crs)

soilgroup_aoi = hydrologic_soil_group.clip(shp_geom)

# Adding to above map
Map.addLayer(soilgroup_aoi, {min:1, max:4, "palette":["green","blue","brown", "white"]}, 'Hydrologic Soil Groups')

# Export to file
geemap.ee_export_image(soilgroup_aoi.clip(shp_geom),
                       path_wd + "soilgroup_aoi.tif",
                       scale = 100,
                       region = shp_geom)

Generating URL ...
Please wait ...
Data downloaded to c:\Work\FW\InVEST\Lukanga_Wetland_Subbasins\GEE_files\soilgroup_aoi.tif


### Topo Data

Note: Can use Hydrosheds OR FabDEM for this!

In [35]:
# https://developers.google.com/earth-engine/datasets/catalog/WWF_HydroSHEDS_03CONDEM
#topo = ee.Image("WWF/HydroSHEDS/03CONDEM") 

# https://samapriya.github.io/awesome-gee-community-datasets/projects/fabdem/
topo_tiles = ee.ImageCollection("projects/sat-io/open-datasets/FABDEM") \
.filterBounds(shp_geom)

# mosaic
topo = topo_tiles.mosaic() \
.setDefaultProjection(project_crs)

topo_aoi = topo.clip(shp_geom)

topoVisParams = {"min":1000,"max":1500,"palette":["green","blue","brown", "white"]}

# Adding to above map
Map.addLayer(topo_aoi, topoVisParams,'Topography')

# Export to file
geemap.ee_export_image(topo_aoi.clip(shp_geom),
                       path_wd + "topo_aoi.tif",
                       scale = 50,
                       region = shp_geom)

Generating URL ...
Please wait ...
Data downloaded to c:\Work\FW\InVEST\Lukanga_Wetland_Subbasins\GEE_files\topo_aoi.tif


## Downloading data for Seasonal Water Yield Module

### Precipitation Data

Note: This uses CHIRPS Monthly data, for ERA5 or other use another code snippet!

In [46]:
# function to clip by shp_geom
def clipbyshape(image):
    return image.clip(shp_geom)


In [47]:
# Import dataset
#prcp = ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY") # https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_LAND_MONTHLY
prcp = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY") # https://developers.google.com/earth-engine/datasets/catalog/UCSB-CHG_CHIRPS_DAILY

# filter for time & date
prcp_filter = prcp \
.filterDate('2002-01-01', '2022-01-01') \
.filterBounds(shp_geom) \
.select('precipitation') # note - if using ERA5, this should be changed to .select('total_precipitation')

# clip by shape
prcp_aoi = prcp_filter.map(clipbyshape)

# function to return monthly totals
def month_average_prcp(mon):
  # filter for all incidences of month
  prcp_mon_all = prcp_aoi.filter(ee.Filter.calendarRange(mon, mon, 'month'))
  # perform monthly averages
  prcp_mon_m_d = prcp_mon_all.reduce(ee.Reducer.mean())
  prcp_mon = prcp_mon_m_d.multiply(30) # convert mm/day to mm/month
  return(prcp_mon)

months = ee.List.sequence(1, 12)
prcp_month = months.map(month_average_prcp)

# Define visualization parameters
prcpVis = {
  'min': 0,
  'max': 300,
  'palette': ['001137', '0aab1e', 'e7eb05', 'ff4a2d', 'e90000'],
}

# Show precipitation map
Map.addLayer(ee.Image(prcp_month.get(3)), prcpVis, 'Precipitation January (mm/month)')

# function to return monthly totals
for mon in range(12):
    mon_act = mon+1
    im = ee.Image(prcp_month.get(mon))
    path = path_wd + "precip_chirps_aoi_" + str(mon_act) + ".tif"
    geemap.ee_export_image(im, path,
                           scale = 5566,
                           region = shp_geom,
                           crs = project_crs
                          )

Generating URL ...
Please wait ...
Data downloaded to c:\Work\FW\InVEST\Lukanga_Wetland_Subbasins\GEE_files\precip_chirps_aoi_1.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Work\FW\InVEST\Lukanga_Wetland_Subbasins\GEE_files\precip_chirps_aoi_2.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Work\FW\InVEST\Lukanga_Wetland_Subbasins\GEE_files\precip_chirps_aoi_3.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Work\FW\InVEST\Lukanga_Wetland_Subbasins\GEE_files\precip_chirps_aoi_4.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Work\FW\InVEST\Lukanga_Wetland_Subbasins\GEE_files\precip_chirps_aoi_5.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Work\FW\InVEST\Lukanga_Wetland_Subbasins\GEE_files\precip_chirps_aoi_6.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Work\FW\InVEST\Lukanga_Wetland_Subbasins\GEE_files\precip_chirps_aoi_7.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Work\FW\InVEST\Luka

### ETo Data

In [48]:
# https://developers.google.com/earth-engine/datasets/catalog/FAO_WAPOR_2_L1_RET_E
ETo = ee.ImageCollection("FAO/WAPOR/2/L1_RET_E")

# filter for dates etc.
ETo_filter = ETo \
.filterDate('2002-01-01', '2022-01-01') \
.filterBounds(shp_geom)

# clip by shape
ETo_aoi = ETo_filter.map(clipbyshape)

# function to return monthly totals
def month_year_average(mon):
  # filter for all incidences of month
  eto_mon_all = ETo_aoi.filter(ee.Filter.calendarRange(mon, mon, 'month'))
  # perform averages for all months
  eto_mon_m_d = eto_mon_all.reduce(ee.Reducer.mean())
  eto_mon = eto_mon_m_d.multiply(3); # convert mm/day to mm/month, add 10 factor!
  return(eto_mon)

months = ee.List.sequence(1, 12)
eto_month = months.map(month_year_average)

# Define visualization parameters
etoVis = {
  'min': 0,
  'max': 500,
  'palette': ['001137', '0aab1e', 'e7eb05', 'ff4a2d', 'e90000'],
}

# Show precipitation map
Map.addLayer(ee.Image(eto_month.get(3)), etoVis, 'ETo selected month (mm/month)')

# function to return monthly totals
for mon in range(12):
    mon_act = mon+1
    im = ee.Image(eto_month.get(mon))
    path = path_wd + "eto_aoi_" + str(mon_act) + ".tif"
    geemap.ee_export_image(im, path,
                           scale = 5566,
                           region = shp_geom,
                           crs = project_crs
                          )

Generating URL ...
Please wait ...
Data downloaded to c:\Work\FW\InVEST\Lukanga_Wetland_Subbasins\GEE_files\eto_aoi_1.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Work\FW\InVEST\Lukanga_Wetland_Subbasins\GEE_files\eto_aoi_2.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Work\FW\InVEST\Lukanga_Wetland_Subbasins\GEE_files\eto_aoi_3.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Work\FW\InVEST\Lukanga_Wetland_Subbasins\GEE_files\eto_aoi_4.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Work\FW\InVEST\Lukanga_Wetland_Subbasins\GEE_files\eto_aoi_5.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Work\FW\InVEST\Lukanga_Wetland_Subbasins\GEE_files\eto_aoi_6.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Work\FW\InVEST\Lukanga_Wetland_Subbasins\GEE_files\eto_aoi_7.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Work\FW\InVEST\Lukanga_Wetland_Subbasins\GEE_files\eto_aoi_8.tif
Generating URL ...
Pleas

### Rainfall events data generation

In [49]:
# function to return mean values
def calcMean(img):
  # gets the mean NDVI for the area in this img
  mean = img.reduceRegion(ee.Reducer.mean(), shp_geom, 5566).get('precipitation')

  # sets the date and the mean NDVI as a property of the image
  return img.set('date', img.date().format()).set('mean', mean)

# applies calcMean() in the collection
col = prcp_aoi.map(calcMean)

# reduces the images properties to a list of lists
values = col.reduceColumns(ee.Reducer.toList(2), ['date', 'mean']).values().get(0)

# type casts the result into a List
lista = ee.List(values)

# converts the list of lists to a Dictionaty
means = ee.Dictionary(lista.flatten())

# export to csv
prcp_csv = path_wd + "precip_daily_series.csv"
geemap.dict_to_csv(means, prcp_csv, by_row=True)


Generating URL ...
Please wait ...
Data downloaded to c:\Work\FW\InVEST\Lukanga_Wetland_Subbasins\GEE_files\precip_daily_series.csv


In [50]:
# Convert output csv to number of rainfall events using pandas
prcp_df = pd.read_csv(prcp_csv, parse_dates=[1])

prcp_df["events"] = prcp_df["value"].ge(1)

#prcp_events = prcp_df.groupby([(prcp_df.name.dt.year), (prcp_df.name.dt.month)])["event"].sum()
monthly_df = prcp_df.groupby(pd.Grouper(freq='MS', key='name')).sum()
#monthly_df = prcp_df.groupby(prcp_df['name'].dt.to_period('m')).sum()
#print(monthly_df)
#print(monthly_df.index)

monthly_events = monthly_df.groupby(monthly_df.index.month)["events"].mean()
monthly_events.index.rename('month', inplace=True)

events_csv = path_wd + "precipitation_events.csv"
monthly_events.to_csv(events_csv)

#print(prcp_events)
#prcp_events_df.groupby(prcp_events_df.name.dt.month)["event"].mean()

## Downloading data for Sediment Delivery Ratio module

### Soil erodibility dataset download

In [51]:
# https://samapriya.github.io/awesome-gee-community-datasets/projects/isric/

soil_props = ['clay', 'silt', 'sand', 'soc']

for prop in soil_props:
    # make paths
    image_loc = "projects/soilgrids-isric/" + prop + "_mean"
    image_out = path_wd + prop + "_aoi.tif"
    # get image from soilgrids
    soil = ee.Image(image_loc)
    soil_reproj = soil \
    .setDefaultProjection(project_crs)
    soil_aoi = soil_reproj.clip(shp_geom)

    # Export to file
    geemap.ee_export_image(soil_aoi.clip(shp_geom),
                           image_out,
                           scale = 500, # this allows for download otherwise too large!
                           region = shp_geom)


Generating URL ...
Please wait ...
Data downloaded to c:\Work\FW\InVEST\Lukanga_Wetland_Subbasins\GEE_files\clay_aoi.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Work\FW\InVEST\Lukanga_Wetland_Subbasins\GEE_files\silt_aoi.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Work\FW\InVEST\Lukanga_Wetland_Subbasins\GEE_files\sand_aoi.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Work\FW\InVEST\Lukanga_Wetland_Subbasins\GEE_files\soc_aoi.tif


### Soil erodibility calculation

In [52]:
from rasterio.plot import show
np.seterr(divide='ignore', invalid='ignore')

# Function for interpolation of 0-30cm depth as described in:
# https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0169748 

def integral_0_30(soil_prop):
    # read rasters using rasterio
    soil = path_wd + soil_prop + "_aoi.tif"
    soil_raster = rasterio.open(soil)
    soil_0_5 = soil_raster.read(1).astype(float)
    soil_5_15 = soil_raster.read(2).astype(float)
    soil_15_30 = soil_raster.read(3).astype(float)
    
    # calculate integral using methodology from https://automating-gis-processes.github.io/CSC18/lessons/L6/raster-calculations.html
    integral = np.empty(soil_0_5.shape, dtype=rasterio.float32)
    check = np.logical_or(soil_0_5 >= 0, soil_5_15 >= 0, soil_15_30 >= 0)
    integral = np.where(check,
                    ((5-0)*(2*soil_0_5) + (15-5)*(2*soil_5_15) + (30-15)*(2*soil_15_30)) / 30 * 0.5,
                    -999 )
    
    return(integral)

# calculating integrals
clay_arr = integral_0_30('clay')/10
silt_arr = integral_0_30('silt')/10
sand_arr = integral_0_30('sand')/10
soc_arr = integral_0_30('soc')/100

# Calculation of soil erodibility from soilgrids data as described in:
# https://www.sciencedirect.com/science/article/pii/S2666765721000107#bib0016
def k_calc(cl,slt,sn,soc):
    sn1 = 1-sn/100
    k_aoi = 0.1317 * (0.2+0.3*np.exp(-0.0256*slt*(1-slt/100))*(slt/(cl+slt))**(0.3)) * (1-(0.25*soc)/(soc+np.exp(3.72-2.95*soc))) * (1-(0.7*sn1)/(sn1+np.exp(22.9*sn1-5.51)))
    return(k_aoi)

check = np.logical_or(clay_arr >= 0, silt_arr >= 0, sand_arr >= 0)
k_aoi = np.where(check, k_calc(clay_arr, silt_arr, sand_arr, soc_arr), -999)

# Write output to TIFF
ras_template = rasterio.open(path_wd + soil_props[1] + "_aoi.tif")
kwargs = ras_template.meta
kwargs.update(
    dtype=rasterio.float32,
    count=1,
    compress='lzw')

k_img_out = path_wd + "soil_erodibility_aoi.tif"

with rasterio.open(k_img_out, 'w', **kwargs) as dst:
    dst.write_band(1, k_aoi.astype(rasterio.float32))


## Running Invest

In [98]:
# coding=UTF-8
# -----------------------------------------------
# Generated by InVEST 3.11.0 on Tue Aug 16 18:38:40 2022
# Model: Seasonal Water Yield

import logging
import sys

import natcap.invest.seasonal_water_yield.seasonal_water_yield
import natcap.invest.utils


ModuleNotFoundError: No module named 'natcap'

In [ ]:

LOGGER = logging.getLogger(__name__)
root_logger = logging.getLogger()

handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter(
    fmt=natcap.invest.utils.LOG_FMT,
    datefmt='%m/%d/%Y %H:%M:%S ')
handler.setFormatter(formatter)
logging.basicConfig(level=logging.INFO, handlers=[handler])

args = {
    'alpha_m': '0.8333',
    'aoi_path': 'C:\\Work\\FW\\InVEST\\Zambia\\InVEST_SWY\\basin_AOI_UTM.shp',
    'beta_i': '1',
    'biophysical_table_path': 'C:\\Work\\FW\\InVEST\\Zambia\\InVEST_SWY\\biophysical_table_aoi_SWY.csv',
    'climate_zone_raster_path': '',
    'climate_zone_table_path': '',
    'dem_raster_path': 'C:\\Work\\FW\\InVEST\\Zambia\\InVEST_SWY\\topo_aoi.tif',
    'et0_dir': 'C:\\Work\\FW\\InVEST\\Zambia\\InVEST_SWY\\ETo',
    'gamma': '1',
    'l_path': '',
    'lulc_raster_path': 'C:\\Work\\FW\\InVEST\\Zambia\\InVEST_SWY\\lu_aoi.tif',
    'monthly_alpha': False,
    'monthly_alpha_path': '',
    'n_workers': '-1',
    'precip_dir': 'C:\\Work\\FW\\InVEST\\Zambia\\InVEST_SWY\\Precip',
    'rain_events_table_path': 'C:\\Work\\FW\\InVEST\\Zambia\\InVEST_SWY\\precipitation_events.csv',
    'results_suffix': 'Zambia',
    'soil_group_path': 'C:\\Work\\FW\\InVEST\\Zambia\\InVEST_SWY\\soilgroup_aoi.tif',
    'threshold_flow_accumulation': '3',
    'user_defined_climate_zones': False,
    'user_defined_local_recharge': False,
    'workspace_dir': 'C:\\Work\\FW\\InVEST\\Zambia\\InVEST_SWY',
}

if __name__ == '__main__':
    natcap.invest.seasonal_water_yield.seasonal_water_yield.execute(args)


In [ ]:
def mean_region(im):
    meanDictionary = im.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=shp_geom,
        scale=5566)
    mean = meanDictionary.get('total_precipitation_mean').getInfo()
    return mean

prcp_means = prcp_aoi.map(mean_region)

In [12]:
im = ee.Image(eto_month.get(2))

meanDictionary = im.reduceRegion(
  reducer=ee.Reducer.max(),
  geometry=shp_geom,
  scale=300
)

print(meanDictionary.get('L1_RET_E_mean').getInfo())

print(range(11))

1318.6600496278052
range(0, 11)


In [30]:
in_js = "c:/Work/FW/InVEST/export_csv.js"
out_py = "c:/Work/FW/InVEST/export_csv.py"
geemap.js_to_python(in_file = in_js, out_file = out_py)

'import ee \nfrom ee_plugin import Map\n\n# Create a function that takes an image, calculates the mean over a\n# geometry and returns the value and the corresponding date as a\n# feature.\ndef createTS(img):\n  date = img.get(\'system_time_start\')\n  value = img.reduceRegion(ee.Reducer.mean(), POI).get(\'LST_Day_1km\')\n  ft = ee.Feature(None, {\'system:time_start\': date,\n                             \'date\': ee.Date(date).format(\'Y/M/d\'),\n                             \'value\': value})\n  return ft\n\n\n# Apply the function to each image in modisLST.\nTS = modisLST.map(createTS)\n\n# Create a graph of the time-series.\ngraph = ui.Chart.feature.byFeature(TS, \'system:time_start\', \'value\')\n\nprint(graph.setChartType("ColumnChart") \\\n           .setOptions({vAxis: {title: \'LST [deg. C]\'},\n                        \'hAxis\': \'{title\': \'Date\'}}))\n\n# Export the time-series as a csv.\nExport.table.toDrive({\'collection\': TS, \'selectors\': \'date, value\'})\n'